# Part 4
This notebook was added later, to produce the values used for Fig. 11 in `https://doi.org/10.1063/5.0139000`

### First load some references

In [ ]:
#r "BoSSSpad.dll"
// #r "..\..\src\L4-application\BoSSSpad\bin\Release\net5.0\BoSSSpad.dll"
using System;
using System.Collections.Generic;
using System.Linq;
using System.IO;
using System.Data;
using System.Globalization;
using System.Threading;
using ilPSP;
using ilPSP.Utils;
using BoSSS.Platform;
using BoSSS.Foundation;
using BoSSS.Foundation.Grid;
using BoSSS.Foundation.Grid.Classic;
using BoSSS.Foundation.IO;
using BoSSS.Solution;
using BoSSS.Solution.Control;
using BoSSS.Solution.GridImport;
using BoSSS.Solution.Statistic;
using BoSSS.Solution.Utils;
using BoSSS.Solution.Gnuplot;
using BoSSS.Application.BoSSSpad;
using BoSSS.Application.XNSE_Solver;
using static BoSSS.Application.BoSSSpad.BoSSSshell;
using BoSSS.Foundation.Grid.RefElements;
using BoSSS.Platform.LinAlg;
using BoSSS.Application.XNSE_Solver.PhysicalBasedTestcases.PrintingNip;
Init();

#### Check if Output should be generated

Only if the output directories were previously created via `Part0_PrintingNip_Setup` output is stored to the subdirectories.  
Otherwise, datatables are stored directly to the working directory and plots displayed in the notebook.  
The plots are not adjusted to look "nice" in the notebook.  
This is not part of the `ValidationTestRunner`.

In [ ]:
static bool GenerateOutput = Directory.Exists(@"./PrintingNip") && Directory.Exists(@"./PrintingNip/Figures") && Directory.Exists(@"./PrintingNip/Files") && Directory.Exists(@"./PrintingNip/Output");

### Init Database etc.

In [ ]:
string ProjectName4Correlation = "PrintingNip_Part1";
string ProjectName = "PrintingNip_Part4";

In [ ]:
BoSSSshell.WorkflowMgm.Init(ProjectName4Correlation);
static var myDb4Correlation = BoSSSshell.OpenDatabase(BoSSSshell.WorkflowMgm.DefaultDatabase.Path);
Console.WriteLine("Initializing : " + myDb4Correlation.Path); // we need to init the static var! otherwise we end up with a reference to the wrong database!

In [ ]:
BoSSSshell.WorkflowMgm.Init(ProjectName);
static var myDb = BoSSSshell.OpenDatabase(BoSSSshell.WorkflowMgm.DefaultDatabase.Path);
Console.WriteLine("Initializing : " + myDb.Path);

In [ ]:
static var myBatch = BoSSSshell.GetDefaultQueue();

In [ ]:
BoSSSshell.WorkflowMgm.SetNameBasedSessionJobControlCorrelation();

### Construct parameterized velocity splines

In [ ]:
double H0 = 1e-5;
double V0 = 1.0;
double P0 = 1e6;

In [ ]:
var sessions = myDb4Correlation.Sessions.Where(s => s.ProjectName == "PrintingNip_Part1").ToArray();
var sV = sessions.Single(s => Convert.ToDouble(s.KeysAndQueries["id:delta"]).ApproxEqual(H0) &  Convert.ToDouble(s.KeysAndQueries["id:P_Diff"]).ApproxEqual(0.0) & Convert.ToDouble(s.KeysAndQueries["id:V_Wall"]).ApproxEqual(V0));
var sP = sessions.Single(s => Convert.ToDouble(s.KeysAndQueries["id:delta"]).ApproxEqual(H0) &  Convert.ToDouble(s.KeysAndQueries["id:P_Diff"]).ApproxEqual(P0) & Convert.ToDouble(s.KeysAndQueries["id:V_Wall"]).ApproxEqual(0.0));

Function to create the velocity profile along the symmetry axis

In [ ]:
using MathNet.Numerics.Interpolation;

In [ ]:
static public CubicSpline ConstructVelocitySpline(ISessionInfo si) {
    DGField VelocityX = si.Timesteps.Last().Fields.Single(f => f.Identification == "VelocityX");
    var grd = (GridData)VelocityX.GridDat;
    EdgeMask em = new EdgeMask(grd, X => Math.Abs(X[1]) < 1e-12); // symmetry axis

    var Spline = Postprocessing.SplineOnEdge(em, VelocityX, 0, out double lB, out double uB);
    return Spline;
}

Evaluate the profile for both simulations and superpose

In [ ]:
var SplineV = ConstructVelocitySpline(sV);
var SplineP = ConstructVelocitySpline(sP);

In [ ]:
Func<double, double, double, double, double> VelocityFunc = (x, h, v, p) => v/V0 * SplineV.Interpolate(x * Math.Sqrt(H0) / Math.Sqrt(h)) + (p * Math.Pow(h,1.5))/(P0 * Math.Pow(H0,1.5)) * SplineP.Interpolate(x * Math.Sqrt(H0) / Math.Sqrt(h));

Set Velocity to be zero at a certain x_stag and calculate the necessary pressure difference

In [ ]:
Func<double, double, double, double> PressureFunc = (x_stag, h, v) => -v/V0 * SplineV.Interpolate(x_stag * Math.Sqrt(H0) / Math.Sqrt(h)) * (P0 * Math.Pow(H0,1.5)) / (Math.Pow(h,1.5) * SplineP.Interpolate(x_stag * Math.Sqrt(H0) / Math.Sqrt(h)));

### Setup Simulations

In [ ]:
double[] deltaS = new double[] {
    0.000001 }; // micro-meter
double[] U = new double[] {
    0.01, 0.03,
    0.1, 0.3,
    1.0, 3.0,
    10.0 };
double[] x_m = new double[] {
    0.001, 0.0025, 0.005,
    0.01, 0.03, 0.05};

// always same
int Res = 10;
int DGdegree = 5;
double R = 0.1;

// set grid to be saved in database
GridFactory.myDb = myDb;

In [ ]:
var controls = new List<XNSE_Control>();

foreach(double delta in deltaS) {
   foreach(double V_wall in U) {
      foreach(double x_stag in x_m) {

         double p = PressureFunc(x_stag, delta, V_wall);

         var C = new XNSE_Control();
         C.DbPath = myDb.Path;
         
         C.SetDGdegree(DGdegree);
         C.SetGrid(GridFactory.GenerateGrid(Res,delta, R));
         C.Paramstudy_CaseIdentification.Add(new Tuple<string, object>("Res", Res));
         C.Paramstudy_CaseIdentification.Add(new Tuple<string, object>("delta", delta));
         C.Paramstudy_CaseIdentification.Add(new Tuple<string, object>("Radius", R));
         C.Paramstudy_CaseIdentification.Add(new Tuple<string, object>("V_Wall", V_wall));
         C.Paramstudy_CaseIdentification.Add(new Tuple<string, object>("P_Diff", p));
         C.Paramstudy_CaseIdentification.Add(new Tuple<string, object>("X_Stag", x_stag));



         C.AddBoundaryValue("wall_walze", "VelocityX", BoundaryValueFactory.Get_VelX(delta, V_wall, R));
         C.AddBoundaryValue("wall_walze", "VelocityY", BoundaryValueFactory.Get_VelY(delta, V_wall, R));
         C.AddBoundaryValue("wall_substrat", "VelocityX", BoundaryValueFactory.Get_VelX(delta, V_wall, R));
         C.AddBoundaryValue("wall_substrat", "VelocityY", BoundaryValueFactory.Get_VelY(delta, V_wall, R));
         C.AddBoundaryValue("pressure_outlet_in", "Pressure", $"X => {p}", false);
         C.AddBoundaryValue("pressure_outlet_out", "Pressure", $"X => {-p}", false);


         C.TimesteppingMode = AppControl._TimesteppingMode.Steady;

         C.PhysicalParameters.rho_A             = 1026.4;
         C.PhysicalParameters.mu_A              = 0.0395;
         C.PhysicalParameters.IncludeConvection = false;
         C.Timestepper_LevelSetHandling         = BoSSS.Solution.XdgTimestepping.LevelSetHandling.None;
         
         C.SessionName = "J" + (20*Res*Res) + "_delta" + delta + "_V" + Math.Round(V_wall, 5) + "_xm" + x_stag;     
         controls.Add(C);
      }
   }
}

In [ ]:
Console.WriteLine(controls.Count);

### Run Simulations

Workaround so we need to deploy the rather large executables only once!

In [ ]:
controls.RunBatch(myBatch, true);

In [ ]:
BoSSSshell.WorkflowMgm.BlockUntilAllJobsTerminate(18000);

Assert that, all sessions are present and all finished successful

In [ ]:
int count = BoSSSshell.wmg.Sessions.Count();
int success = BoSSSshell.wmg.Sessions.Where(s => s.SuccessfulTermination).Count();

if(count != controls.Count() || count != success){
    throw new ApplicationException("Not all simulations calculated or finished successful!");
}

### Load Simulations

In [ ]:
var sessions = myDb.Sessions.Where(s => s.ProjectName == ProjectName && s.SuccessfulTermination).ToArray();

In [ ]:
sessions.Count()

### Setup Post-Processing

In [ ]:
BoSSSshell.WorkflowMgm.AdditionalSessionTableColums.Clear();

In [ ]:
BoSSSshell.WorkflowMgm.AdditionalSessionTableColums.Add("dPdXatStagnationPoint", Postprocessing.dPdXatStagnationPoint);

### Create Datatable

In [ ]:
DataTable tab;
if(GenerateOutput){
    if(!File.Exists("./PrintingNip/Files/PrintingNip-Part4Export.csv")){
        tab = sessions.GetSessionTable(BoSSSshell.WorkflowMgm.AdditionalSessionTableColums.Select(kv => new Tuple<string, Func<ISessionInfo, object>>(kv.Key, kv.Value)).ToArray());
        tab.ToCSVFile("./PrintingNip/Files/PrintingNip-Part4Export.csv", ColSep: '%');
    } else{
        tab = TableExtensions.FromCSVFile("./PrintingNip/Files/PrintingNip-Part4Export.csv", ColSep: '%');
    }
} else {
    // in this case always recreate the datatable
    tab = sessions.GetSessionTable(BoSSSshell.WorkflowMgm.AdditionalSessionTableColums.Select(kv => new Tuple<string, Func<ISessionInfo, object>>(kv.Key, kv.Value)).ToArray());
    tab.ToCSVFile("./PrintingNip-Part4Export.csv", ColSep: '%');     
}

### Export Datatable

In [ ]:
string[] Columns = new string[] {"dPdXatStagnationPoint"};
string[] Labels = new string[] {@"$\\\frac{\\\partial p}{\\\partial x}_{stag}$"};
string[] Units = new string[] {@"$\\\left[\\\frac{N}{m^3}\\right]$"}; // some escape character nonsense

In [ ]:
var tab2export = tab.ExtractColumns(Columns.ToList().Prepend("id:V_Wall").Prepend("id:P_Diff").Prepend("id:Radius").Prepend("id:delta").Prepend("id:X_Stag").Prepend("SessionName").ToArray());
if(GenerateOutput){
    tab2export.ToHTMLFile("PrintingNip-Part4.html", Path.GetFullPath("./PrintingNip/Files"));
    tab2export.ToCSVFile("./PrintingNip/Files/PrintingNip-Part4.csv", ColSep: ';');
} else {
    tab2export.ToHTMLFile("PrintingNip-Part4.html", Path.GetFullPath("./"));
    tab2export.ToCSVFile("./PrintingNip-Part4.csv", ColSep: ';');
}